In [13]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import glob
reload(ld)

<module 'sib_ldsc_z' from 'c:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [14]:
np.random.seed(123)

# N = int(100)
# S_size = int(N/2)
# S = np.array([np.array([[.5, 0], [0, .8]]),
#     np.array([[0.5, 0], [0, 0.8]])] * S_size )/N
# V = np.identity(2) * 0.5

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])


# N = int(1e4)
# S = np.array([[[1e-4, -5 * 1e-5, -4 * 1e-5], [-5 * 1e-5, 1e-4, -4 * 1e-5], [-5 * 1e-5, -4 * 1e-5, 1e-4]]] * N)
# V = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])

model = ld.sibreg(S = S)
model.simdata(V/N, N, simld = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for LD Scores given. Generating a vector of ones for l
No value for effective number of loci is given. Using total number of loci instead
Simulated LD scores!


In [15]:
# Testing derivatives
aderiv = ld._grad_ll_v(ld.Vmat2V(V, N), model.z[0, :], model.S[0],  model.l[0], N)

nderiv = ld._num_grad_V(ld.Vmat2V(V, N), model.z[0, :], model.S[0], model.l[0],  N)

np.allclose(aderiv, nderiv)

True

In [16]:
# solving
output, result = model.solve()
print(result)

No initial guess provided.
Making Method of Moments Guess
Initial estimate: [1.55430175e-04 1.47910632e-04 8.09808861e-05]
      fun: 37189.05234751942
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00462407, -0.00017175,  0.00281469])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 12
      nit: 9
   status: 0
  success: True
        x: array([0.51801914, 0.49265043, 0.53215488])


In [17]:
output

{'v1': 0.5180191426965134,
 'v2': 0.49265042589569286,
 'r': 0.5321548816445631,
 'std_err_mat': array([[0.02100374,        nan,        nan],
        [       nan, 0.0203435 ,        nan],
        [       nan,        nan, 0.03004591]])}

In [18]:
print("Real Parameters:")
print(f"v1 = {V[0, 0]}")
print(f"v2 = {V[1, 1]}")
print(f"r = {V[0, 1]/np.sqrt(V[0, 0] * V[1, 1])}")

Real Parameters:
v1 = 0.5
v2 = 0.5
r = 0.5


In [21]:
# Monte carlo simulation comparing JKSE to 
# Inverse Hessian Root standard error

nsim = 100
np.random.seed(123)
jkse_est = np.zeros((nsim, 3))
invhess_est = np.zeros((nsim, 3))

for i in range(nsim):

    # Simulating
    N = int(1e4)
    S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
    V = np.array([[0.5, 0.25], [0.25, 0.5]])

    model = ld.sibreg(S = S)
    model.simdata(V/N, N, simld = True)

    # solving
    output, _ = model.solve()

    invhess_est[i, :] = np.diag(output['std_err_mat'])
    jkse_est[i, :] = model.jackknife_se(blocksize = int(N/100))


lated LD scores!
No initial guess provided.
Making Method of Moments Guess
Initial estimate: [1.56225431e-04 1.55132265e-04 7.54660275e-05]
Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for LD Scores given. Generating a vector of ones for l
No value for effective number of loci is given. Using total number of loci instead
Simulated LD scores!
No initial guess provided.
Making Method of Moments Guess
Initial estimate: [1.42488832e-04 1.50746350e-04 7.78408999e-05]
Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for LD Scores given. Generating a vector of ones for l
No value for effective number of loci is given. Using total number of loci instead
Simulated LD scores!
No initial guess provided.
Making Method of Moments Guess
Initial estimate: [1.46613862e-04 1.41693511e-04 7.32587572e-05

KeyboardInterrupt: 